In [24]:
import numpy as np
from typing import Tuple
from keras.api.models import Sequential
from keras.api.layers import Dense
from keras.api.optimizers import Adam
from collections import deque
from IPython.display import IFrame
import pyautogui
import random

from Config.sansConfig import ScreenConfig

In [15]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0   # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# You would need to modify the game to:
# 1. Provide state observations (player position, bullet positions, etc.)
# 2. Accept discrete actions (move left, right, up, down, etc.)
# 3. Calculate rewards (survival time, damage dealt, etc.)

In [2]:
from IPython.display import IFrame
IFrame(src="https://jcw87.github.io/c2-sans-fight/", width=800, height=600)  # Replace with your hosted URL

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

# Launch Chrome and load the game
driver = webdriver.Chrome()
driver.get("https://jcw87.github.io/c2-sans-fight/")  # Hosted game URL
time.sleep(2)  # Wait for game to load

# Focus the game window
driver.find_element("tag name", "body").click()

In [36]:
def get_google_rect() -> dict[str : Tuple[int, int]]:
        game_rect : dict[str : Tuple[int, int]] = {}

        browser_window = pyautogui.getWindowsWithTitle("Bad Time Simulator")[0]  # or "Chrome" if that's what appears in the title
        
        # Get window coordinates
        x, y = browser_window.left, browser_window.top
        width, height = browser_window.width, browser_window.height
        
        # Adjust these values to capture just the game area (not the browser UI)
        game_x = x + 10  # Add padding to skip browser borders
        game_y = y + 100  # Skip address bar and tabs
        game_width = width - 20
        game_height = height - 120

        game_rect["corrdinates"] = [game_x, game_y]
        game_rect["scale"] = [game_width, game_height]

        return game_rect

In [37]:
import cv2
import numpy as np
from PIL import ImageGrab


def get_game_state():
    # Capture the game area (adjust coordinates to match the IFrame)
    rect = get_google_rect()
    x = rect["corrdinates"][0]
    y = rect["corrdinates"][1]

    width = rect["scale"][0]
    height = rect["scale"][1]

    screenshot = ImageGrab.grab(bbox=(x, y, x+width, y+height))  # Replace with your IFrame position
    screenshot = np.array(screenshot)
    screenshot = cv2.cvtColor(screenshot, cv2.COLOR_RGB2GRAY)  # Convert to grayscale
    screenshot = cv2.resize(screenshot, (84, 84))  # Resize for DQN input
    return screenshot

state = get_game_state()

In [ ]:
def player_hit(state):
    # Check if player's red heart pixel is missing (got hit)
    return state[player_y, player_x] != RED_VALUE

In [30]:
def take_action(action):
    body = driver.find_element("tag name", "body")
    if action == 0:    # Move left
        body.send_keys(Keys.ARROW_LEFT)
    elif action == 1:  # Move right
        body.send_keys(Keys.ARROW_RIGHT)
    elif action == 2:  # Move up
        body.send_keys(Keys.ARROW_UP)
    elif action == 3:  # Move down
        body.send_keys(Keys.ARROW_DOWN)
    elif action == 4:  # Attack (Z key)
        body.send_keys("z")
    elif action == 5:  # Do nothing
        pass  # No action, do nothing

In [ ]:
def get_reward(prev_state, current_state):
    # Check if player got hit (pixel color change)
    if player_hit(current_state):
        return -10  # Penalty for getting hit
    else:
        return 1   # Reward for surviving each frame

In [ ]:
from keras.api.models import Sequential
from keras.api.layers import Dense, Conv2D, Flatten
from collections import deque
import random

class DQNAgent:
    def __init__(self):
        self.state_size = (84, 84, 1)  # Grayscale image dims
        self.action_size = 5  # [Left, Right, Up, Down, Attack]
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Conv2D(32, (8, 8), strides=4, activation='relu', input_shape=self.state_size))
        model.add(Conv2D(64, (4, 4), strides=2, activation='relu'))
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer='adam')
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = np.expand_dims(state, axis=0)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def train(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state, verbose=0)[0])
            target_f = self.model.predict(state, verbose=0)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > 0.01:
            self.epsilon *= 0.995

agent = DQNAgent()

In [ ]:
batch_size = 32
episodes = 1000

for e in range(episodes):
    driver.refresh()  # Reset game
    time.sleep(2)
    state = get_game_state()
    state = np.expand_dims(state, axis=-1)  # Add channel dim
    total_reward = 0
    
    while True:
        action = agent.act(state)
        take_action(action)
        time.sleep(0.05)  # Adjust for game speed
        
        next_state = get_game_state()
        next_state = np.expand_dims(next_state, axis=-1)
        reward = get_reward(state, next_state)
        done = check_game_over(next_state)  # Implement this
        
        agent.memory.append((state, action, reward, next_state, done))
        total_reward += reward
        state = next_state
        
        if done:
            print(f"Episode: {e}, Reward: {total_reward}")
            break
        
        if len(agent.memory) > batch_size:
            agent.train(batch_size)

In [1]:
def player_hit(state):
    # Check if player's red heart pixel is missing (got hit)
    return state[player_y, player_x] != RED_VALUE

In [ ]:
import cv2
import numpy as np
from PIL import ImageGrab
import pyautogui

def get_game_state():
    # Get the position of the browser window (you may need to adjust this)
    try:
        # Try to locate the game window on screen (requires the browser to be visible)
        browser_window = pyautogui.getWindowsWithTitle("Bad Time Simulator")[0]  # or "Chrome" if that's what appears in the title
        
        # Get window coordinates
        x, y = browser_window.left, browser_window.top
        width, height = browser_window.width, browser_window.height
        
        # Adjust these values to capture just the game area (not the browser UI)
        game_x = x + 10  # Add padding to skip browser borders
        game_y = y + 100  # Skip address bar and tabs
        game_width = width - 20
        game_height = height - 120
        
        # Capture the game area
        screenshot = ImageGrab.grab(bbox=(game_x, game_y, game_x+game_width, game_y+game_height))
    except:
        # Fallback: capture entire screen if window detection fails
        screenshot = ImageGrab.grab()
    
    # Process the image
    screenshot = np.array(screenshot)
    screenshot = cv2.cvtColor(screenshot, cv2.COLOR_RGB2GRAY)  # Convert to grayscale
    screenshot = cv2.resize(screenshot, (84, 84))  # Resize for DQN input
    return screenshot

# Test it
state = get_game_state()
cv2.imshow('Game Capture', state)
cv2.waitKey(0)
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'pyautogui'